<a href="https://colab.research.google.com/github/alefram/blog-notebooks/blob/main/cuda_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA setup for experiment in C++ and Python